In [ ]:
from crewai import Agent, Crew, Task
from crewai_tools import CodeInterpreterTool
from langchain_groq import ChatGroq

In [ ]:
from typing import List


In [ ]:
from utils import get_openai_api_key, get_serper_api_key, get_groq_api_key

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv()

In [ ]:


llm = ChatGroq(
    model="llama-3.1-70b-versatile"
)

In [ ]:
understand_problem = Agent(
    role='Understand Problem',
    goal=("""Understand the problem statement and identify the key components and list them down 
          for the next agent to work upon. The question will be given as a statement clearly defining
          the *constraints*, *Input Format*, *Output Format* and *Sample Explanation*. You are to break down
          all the needs of the problem and list them in an understandable format.
          
          Here is an example of the problem that will be given to you:

          You've found a solution to an implementation-heavy geometry problem that requires typing out N lines of code. Annoyingly, you only have a P% chance of typing out any given line without a mistake, and your code will only be accepted if all N lines are correct. The chance of making a mistake in one line is independent of the chance of making a mistake in any other line.

          You realize there might be a solution which only requires N-1 lines (each also having a P% chance of being typed correctly). However, instead of thinking about that, you could also just type out the N-line solution more carefully to increase P. How much would P have to increase to yield the same chance of success as needing to type one fewer line of code?

          # Constraints
          1 ≤ T ≤ 100
          2 ≤ N ≤ 1,000
          1 ≤ P ≤ 99

          # Input Format
          Input begins with an integer T, the number of test cases. Each case is a single line containing the integers N and P.

          # Example Input Format
          4
          3 10
          2 50
          13 37
          950 95

          # Output Format
          For the ith test case, print "Case #i: " followed by how much higher P would need to be to make spending your time typing carefully be as successful as typing one line fewer with your original P.

          Your answer will be accepted if it is within an absolute or relative error of 10⁻⁶.

          # Sample Explanation
          In the first case, you initially need to type 2 lines. You can either type just 1 line with a 50% success rate, or you could improve your typing accuracy to √50% ≈ 70.710678%, at which point you'd have a √50%^2 = 50% chance of successfully typing the original 2 lines. So you would need to increase P by 70.710678 - 50 = 20.710678 for both approaches to have an equal chance of success.
    """)

          ,
    backstory=("""
                You are a Phd level Mathematician whose approach to
                handling tough mathematical problems and identiifying the core underlying needs
                is unmatched. You have a keen eye for detail and a passion for understanding the
                underlying principles and the underlying forces that lead to a problem solution.
                You have a deep understanding of the mathematical theory and have a good
                understanding of the underlying principles and the underlying forces that lead to a
                problem solution. You are also a good communicator and a great problem solver.
              
               """),
    # tools=tools,
    verbose=True,
    allow_delegation=False
)

In [ ]:
understand_problem_task = Task(
    description="""
    Analyze the given {problem} statement to identify and extract all relevant components for the next stages of processing.
    You are required to:
    1. Parse the *constraints* provided in the {problem} to define the boundaries within which the solution must operate.
    2. Identify the *input format* specifications, including the types and structure of data that will be provided.
    3. Define the *output format* based on the {problem} requirements, ensuring that it aligns with the specified solution expectations.
    4. Break down the {problem} into key steps that will be handled in subsequent stages.
    5. Provide detailed explanations or clarifications on any ambiguities in the {problem} statement.
    6. Consider edge cases and potential issues that might arise in the solution process and document them for later agents.
    """,
    expected_output="""
    A well-structured and detailed breakdown of the {problem} that includes:
    1. A clear list of constraints.
    2. A description of the input format with examples.
    3. A description of the expected output format with examples.
    4. A list of steps for solving the {problem}, organized for further agents to implement.
    5. A list of edge cases and ambiguous areas to be addressed in future stages.
    6. A .md file carrying all this detail
    """,
    # tools=tasks['planning']['tools'],
    output_file="understanding.md",
    async_execution=False,  # Synchronous execution for better coordination
    # Assigning the 'understand_problem' agent to execute this task
    agent=understand_problem
)


In [ ]:
math_crew = Crew(
    agents=[understand_problem],
    tasks=[understand_problem_task],
    verbose=True,
    
)


In [ ]:
math_crew.kickoff(inputs={"problem": "Mark Twain"})